In [1]:
# 인터파크 투어 사이트에서 여행지를 입력후 검색 -> 잠시후 -> 결과
# 로그인시 PC 웹 사이트에서 처리가 어려울 경우 -> 모바일 로그인 진입
# 모듈을 가져오기
# pip install selenium
# pip install bs4
# pip install pymysql

In [2]:
##DbMgr
# 디비처리, 연결, 해제, 검색어 가져오기, 데이터 삽입

import pymysql as my

class DBHelper:
    '''
    멤버변수 : 커넥션
    '''
    conn = None
    '''
    생성자
    '''
    def __init__(self):
        self.db_init()
        '''
        멤버 함수
        '''
    def db_init(self):
        self.conn = my.connect(host='localhost',
                                user='root',
                                password='1234',
                                db='pythonDB',
                                charset='utf8',
                                cursorclass=my.cursors.DictCursor
            
        )
    def db_free(self):
        if self.conn:
            self.conn.close
                
    # 검색 키워드 가져오기 => 웹에서 검색
    def db_selectKeyword(self):
        #커서 오픈
        #with => 닫기 처리를 자동으로 처리해준다 => I/O 많이 사용
        rows = None
        with self.conn.cursor() as cursor:
            sql = "SELECT * FROM tbl_keyword;"
            cursor.execute(sql)
            rows = cursor.fetchall()
            print(rows)
        return rows
        
    def db_insertCrawlingData(self, title, price, area, contents, keyword):
        with self.conn.cursor() as cursor:
            sql = '''
            insert into `tbl_crawlingdata`
            (title, price, area, contents,keyword)
            values ( %s, %s, %s, %s, %s )
            '''
            cursor.execute(sql, (title, price, area, contents, keyword))
        self.conn.commit()

In [19]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By

# 명시적 대기를 위해
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from DbMgr
#import DBHelper as Db
import pymysql as my
import time
#from Tour import TourInfo

In [20]:
#사전에 필요한 정보를 로드 => DB혹은 쉘, 베치 파일에서 인자로 받아서 셋팅
db      =DBHelper()
main_url='https://tour.interpark.com/'
keyword = ('독일')

#  상품 정보를 담는 리스트 (TourInfo 리스트)
tour_list = [ ]

In [21]:
#드라이버 로드
driver = wd.Chrome(executable_path='chromedriver.exe')

In [22]:
# 차후 -> 옵션 부여하여 (프록시, 에이전트 조작, 이미지를 배제)
# 크롤링을 오래돌리면 => 임시파일들이 쌓인다 -> 템포 파일 삭제

In [23]:
# 사이트 접속 (get)
driver.get(main_url)

In [24]:
#검색창을 찾아서 검색어 입력
# id : SearchGNBText
# 찾기                                     입력
driver.find_element_by_id('SearchGNBText').send_keys(keyword)

In [25]:
# 수정할경우 => 뒤에 내요잉 붙어버림 => .clear() -> send_keys('내용')
# 검색버튼을 클릭
driver.find_element_by_css_selector('button.search-btn').click()

In [26]:
# 잠시대기 => 페이가 로드되고 나서 즉각적으로 데이터를 획득 하는 행위는
# 명시적 대기 => 특정 요소가  로케이트(발견될때까지) 대기
try:
    element = WebDriverWait(driver, 10).until(
        #지정한 한개 요소가 올라오면 웨이트 종료
        EC.presence_of_element_located((By.CLASS_NAME, 'oTravelBox'))
    )
except Exception as e:
    print('오류발생', e)

In [27]:
# 암묵적 대기 => DOM이 다 로드 될때까지 대기하고 먼저 로드되면 바로 진행
# 요소를 찾을 특정 시간 동안 DOM 풀링을 지시 예를 들어 10초이내라도 발견되면 바로 진행
driver.implicitly_wait(10)

In [28]:
# 절대적 대기 => time.sleep(10) -> 클라우드 페어(디도스 방어 솔루션)

In [29]:
# 더보기 눌러서=>게시판 진입
driver.find_element_by_css_selector('.oTravelBox>.boxList>.moreBtnWrap>.moreBtn').click()

In [30]:
# 상품 정보를 담는 클래스
class TourInfo:
    
    # 멤버변수 (실제 컬럼보다는 작게 세팅했음)
    self = ''
    title = ''
    price = ''
    area = ''
    link = ''
    img = ''
    contents = ''
    # 생성자
    def __init__(self, title, price, area, link,img, contents=None):
        self.title = title
        self.price = price
        self.area = area
        self.link = link
        self.img = img
        self.contents = contents

In [31]:
# 게시판에서 데이터를 가져올때
# 데이터가 많으면 세션(혹시 로그인을 해서 접근되는 사이트일 경우) 관리
# 특정 단위별로 로그아웃 로그인 계속 시도
# 특정 게시물이 사라질 경우 => 팝업 발생(없는...) => 팝업 처리 검토
# 게시판 스캔시 => 임계점을 모름!!
# 게시판 스캔 => 메타 정보 획득 => loop를 돌려서 일괄적으로 방문 접근 처리

In [32]:
# searchModule.SetCategoryList(1, '') 스크립트 실행
# 25는 임시값, 게시물을 넘어갔을때 현상을 확인차
for page in range(1,2): #25):
    try:
        #자바스크립트 구동하기         
        driver.execute_script("searchModule.SetCategoryList(%s, '')" % page)
        time.sleep(2)
        print("%s 페이지 이동" % page)
        ####################################################################3333
        #여러 사이트에서 정보를 수집할 경우 공통 정보 정의 단계 필요
        # 상품명, 코멘트, 기간1, 기간2, 가격, 평점, 썸네일, 링크(상품상세정보)
        boxItems = driver.find_elements_by_css_selector('.oTravelBox>.boxList>li')
#       상품 하나하나 접근
        for li in boxItems:
            # 이미지를 링크 값을 사용할 것인가? 직접 다운로드 해서 우리 서버에 업로드(ftp) 할 것인다?
            print('썸네일', li.find_element_by_css_selector('img').get_attribute('src') )
            print('URL', li.find_element_by_css_selector('a').get_attribute('onclick') )
            print('상품명', li.find_element_by_css_selector('h5.proTit').text )
            print('코멘트', li.find_element_by_css_selector('.proSub').text )
            print('가격', li.find_element_by_css_selector('.proPrice').text )
#             info-row
#             info
            for info in li.find_elements_by_css_selector('.info-row .proInfo'):
                print(info.text)
            print('='*100)
            
            #데이터 모음
            #li.find_elements_by_css_selector('.info-row .proInfo')[1].text
            #데이터가 부족하거나 없을수도 있으므로 직접 인덱스로 표현은 위험성이 있음
            obj =  TourInfo(
                li.find_element_by_css_selector('h5.proTit').text,
                li.find_element_by_css_selector('.proPrice').text,
                li.find_elements_by_css_selector('.info-row .proInfo')[1].text,
                li.find_element_by_css_selector('a').get_attribute('onclick'),
                li.find_element_by_css_selector('img').get_attribute('src')
            )
            tour_list.append(obj)
    except Exception as e1:
        print('오류', e1)
print (tour_list, len(tour_list))

1 페이지 이동
썸네일 http://tourimage.interpark.com/product/tour/00161/A30/280/A3014604_8_990.jpg
URL searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3014604','')
상품명 [스위스/이태리9일] 1급UP! 리기정상식+몽트뢰/프라이부르크_OZ/KE
코멘트 국적기 직항/루체른/몽트뢰/프라이부르크
가격 1,650,000 원~
여행 기간 : 7박9일독일-스위스-이탈리아
출발 가능 기간 : 2019.09.17~2020.07.16
평점 8.8
53개의 상품평
썸네일 http://tourimage.interpark.com/product/tour/00161/B30/280/B3011186_6_163.jpg
URL searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=B3011186','')
상품명 [베를린/드레스덴/프라하 자유 7일] 시내중심호텔+도시간이동열차
코멘트 요즘 뜨는 여행지!
가격 1,590,000 원~
여행 기간 : 5박7일베를린2-드레스덴1-프라하2
출발 가능 기간 : 2019.09.16~2020.06.30
평점 8.3
2개의 상품평
썸네일 http://tourimage.interpark.com/product/tour/00161/B30/280/B3010739_6_487.jpg
URL searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=B3010739','')
상품명 [베를린5일]ONE CITY★베를린 자유여행 5일
코멘트 요즘 Hip한 도시!
가격 1,290,000 원~
여행 기간 : 3박5일베를린3
출발 가능 기간 : 2019.09.16~2020.06.30
평점 9.3
2개의 상품평
썸네일 http://tourima

In [33]:
# 수집한 정보 개수를 루프 => 페이지 방문 => 콘텐츠 획득(상품상세정보) => 디비
for tour in tour_list:
    #tour => TourInfo
    print(type(tour))
    #링크 데이터에서 실데이터 획득
    # 분해
    arr = tour.link.split(',')
    if arr:
        #대체
        link = arr[0].replace('searchModule.OnClickDetail(','')
        #슬라이싱 => ''을 제거
        detail_url = link[1:-1]
        #print(detail_url)
        #상세 페이지 이동 : ulr 값이 완성된 형태인지 확인(http~)
        driver.get(detail_url)
        time.sleep(1)
        # pip install bs4
        # 현재 페이지를 beautifulsoup 의 DOM 으로 구성
        soup = bs(driver.page_source, 'html.parser')
        #ㅎ 현재 상세 정보 페이지에서 스케줄 정보 획득
        data = soup.select('.tip-cover')
        print(type(data), len(data), data[0].contents)
        # 디비 입력 => pip install pymysql
        content_final = ''
        for c in data[0].contents:
            contnet_final = str(c)

        # 컨텐츠 내용에 따라 전처리
        db.db_insertCrawlingData(
            tour.title,
            tour.price,
            tour.area,
            contnet_final,
            keyword
        )

<class '__main__.TourInfo'>
<class 'list'> 1 ['\n', ' 인터파크 혜택 ', '\n', <div class="default-section special" id="divGoodsSpecialInfo">
<h3>인터파크 혜택</h3>
<p align="center" style="TEXT-ALIGN: center"><a href="http://tour.interpark.com/event/event_view.aspx?seq=13033" target="_self"><img src="http://tourimage.interpark.com/BBS/Tour/FckUpload/201907/6369758515322929650.jpg"/></a></p><span style="COLOR: #0075c8"><span style="COLOR: #0075c8"><span style="COLOR: #0075c8"><span style="COLOR: #0075c8"><img border="0" height="555" rheight="555" rwidth="910" src="http://tourimage.interpark.com/BBS/Tour/FckUpload/201812/6368153103888292890.png" style="HEIGHT: 555px; WIDTH: 910px; rwidth: 910px; rheight: 555px" width="910"/><span style="COLOR: #0075c8"><span style="COLOR: #0075c8"><span style="COLOR: #0075c8"><span style="COLOR: #0075c8"><span style="COLOR: #0075c8"><span style="COLOR: #0075c8"><span style="COLOR: #0075c8"><span style="COLOR: #0075c8"><span style="COLOR: #0075c8"><span style="COLOR: 

<class 'list'> 1 ['\n', ' 인터파크 혜택 ', '\n', <div class="default-section special" id="divGoodsSpecialInfo">
<h3>인터파크 혜택</h3>
<p style="LINE-HEIGHT: 180%"><font color="#0000ff"><span style="FONT-FAMILY: 굴림,gulim"><font color="#ff0000"><font size="3"><span style="FONT-FAMILY: '나눔고딕',NanumGothic,Sans-serif; COLOR: #091fe9">●항공 실시간 예약 상품으로 상품가격이 변동 될 수 있습니다. 예약 후 담당자의 확정견적을 받으셔야 합니다. </span><br/><span style="FONT-FAMILY: '나눔고딕',NanumGothic,Sans-serif; COLOR: #091fe9">●일정 연장 등 맞춤으로 변경 가능한 자유여행 상품!!</span></font></font></span></font></p><p> </p><p> </p><p align="center" style="WHITE-SPACE: normal; WORD-SPACING: 0px; TEXT-TRANSFORM: none; COLOR: rgb(100,100,100); TEXT-ALIGN: center; FONT: 12px/18px 돋움, dotum, san-serif; MARGIN: 0px; WIDOWS: 1; LETTER-SPACING: normal; BACKGROUND-COLOR: rgb(255,255,255); TEXT-INDENT: 0px; -webkit-text-stroke-width: 0px; font-stretch: normal"><font color="#3366ff" style="LINE-HEIGHT: 1.2em"><u><font color="#000000" style="LINE-HEIGHT: 1.2em"><span class="Apple-con

<class 'list'> 1 ['\n', ' 인터파크 혜택 ', '\n', <div class="default-section special" id="divGoodsSpecialInfo">
<h3>인터파크 혜택</h3>
<strong><font color="#0000ff"><strong style="FONT-SIZE: 12px; FONT-FAMILY: 돋움, dotum, san-serif; FONT-VARIANT: normal; WHITE-SPACE: normal; TEXT-TRANSFORM: none; WORD-SPACING: 0px; COLOR: rgb(239,0,124); FONT-STYLE: normal; WIDOWS: 1; LETTER-SPACING: normal; LINE-HEIGHT: 1.2em; BACKGROUND-COLOR: rgb(255,255,255); TEXT-INDENT: 0px; -webkit-text-stroke-width: 0px"><span style="COLOR: rgb(239,0,124); LINE-HEIGHT: 1.2em"><strong style="LINE-HEIGHT: 1.2em"><p> </p><p><font color="#000000"><font color="#000000"><span class="Apple-converted-space" style="LINE-HEIGHT: 1.2em"><span style="FONT-FAMILY: 굴림,gulim"><font color="#ff0000"><font size="3"><span style="FONT-FAMILY: '나눔고딕',NanumGothic,Sans-serif; COLOR: rgb(9,31,233)">●항공 실시간 예약 상품으로 상품가격이 변동 될 수 있습니다. 예약 후 담당자의 확정견적을 받으셔야 합니다. </span><br/><span style="FONT-FAMILY: '나눔고딕',NanumGothic,Sans-serif; COLOR: rgb(9,31,233)"

<class 'list'> 1 ['\n', ' 인터파크 혜택 ', '\n', <div class="default-section special" id="divGoodsSpecialInfo">
<h3>인터파크 혜택</h3>
<font color="#003366" style="LINE-HEIGHT: 1.2em"><strong style="LINE-HEIGHT: 1.2em"><span style="COLOR: rgb(239,0,124); LINE-HEIGHT: 1.2em"></span></strong></font><p> </p><p><font color="#0000ff"><font color="#ff0000"><strong><span id="husky_bookmark_start_1459301077540"></span></strong></font></font><font color="#000000" style="LINE-HEIGHT: 1.2em"><span class="Apple-converted-space" style="LINE-HEIGHT: 1.2em"><span style="FONT-FAMILY: 굴림,gulim"><font color="#ff0000"><font size="3"><span style="FONT-FAMILY: '나눔고딕',NanumGothic,Sans-serif; COLOR: rgb(9,31,233)">●항공 실시간 예약 상품으로 상품가격이 변동 될 수 있습니다. 예약 후 담당자의 확정견적을 받으셔야 합니다. </span><br/><span style="FONT-FAMILY: '나눔고딕',NanumGothic,Sans-serif; COLOR: rgb(9,31,233)">●일정 연장 등 맞춤으로 변경 가능한 자유여행 상품!!</span></font></font></span></span></font> </p><p align="left" style="TEXT-ALIGN: left"> </p><p align="left" style="TEXT-ALIGN: l

<class 'list'> 1 ['\n', ' 인터파크 혜택 ', '\n', <div class="default-section special" id="divGoodsSpecialInfo">
<h3>인터파크 혜택</h3>
<p> </p><p><font color="#0000ff"><font color="#ff0000"><strong><span id="husky_bookmark_start_1459301077540"></span></strong></font></font><font color="#000000" style="LINE-HEIGHT: 1.2em"><span class="Apple-converted-space" style="LINE-HEIGHT: 1.2em"><span style="FONT-FAMILY: 굴림,gulim"><font color="#ff0000"><font size="3"><span style="FONT-FAMILY: '나눔고딕',NanumGothic,Sans-serif; COLOR: rgb(9,31,233)">●항공 실시간 예약 상품으로 상품가격이 변동 될 수 있습니다. 예약 후 담당자의 확정견적을 받으셔야 합니다. </span><br/><span style="FONT-FAMILY: '나눔고딕',NanumGothic,Sans-serif; COLOR: rgb(9,31,233)">●일정 연장 등 맞춤으로 변경 가능한 자유여행 상품!!</span></font></font></span></span></font></p><p>  </p><p><img alt="" src="http://tourimage.interpark.com/Sites/Bn/2013/07/6350830166777889263.jpg"/><img alt="" src="http://tourimage.interpark.com/Sites/Bn/2013/07/6350830181390389266.jpg"/><img border="0" src="http://tourimage.interpark.com/

In [34]:
# 단독으로 수행 시에만 작동 => 테스트코드로 삽입해서 사용
if __name__=='__main__':
    db = DBHelper()
    print(db.db_selectKeyword())
    print(db.db_insertCrawlingData('1', '2', '3', '4', '5'))
    db.db_free()

[{'keyword': '바르셀로나'}, {'keyword': '베를린'}, {'keyword': '파리'}]
[{'keyword': '바르셀로나'}, {'keyword': '베를린'}, {'keyword': '파리'}]
None


In [35]:
# 종료
driver.close()
driver.quit()
#시스템 종료
import sys
sys.exit()

SystemExit: 

C:\anaconda\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
